In [ ]:
from __future__ import division
import pickle
import os
import types
import random
import uuid
from copy import deepcopy as copy

import gym
from gym import spaces
from gym.envs.classic_control import rendering
import numpy as np
import tensorflow as tf
from scipy.misc import logsumexp

In [ ]:
from matplotlib import pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
%matplotlib inline

In [ ]:
import matplotlib as mpl
mpl.rc('savefig', dpi=300)
mpl.rc('text', usetex=True)

In [ ]:
data_dir = os.path.join('data', '2.0-tabular-irl')

In [ ]:
sess = tf.Session()

create envs

In [ ]:
gw_size = 7
n_train_tasks = 10
n_act_dim = 4
n_obs_dim = gw_size**2 + 1
succ_rew_bonus = 1
crash_rew_penalty = -1
gamma = 0.99
max_ep_len = 100

In [ ]:
is_succ = lambda r: r[-1][2] > succ_rew_bonus / 2
is_crash = lambda r: r[-1][2] < crash_rew_penalty / 2

In [ ]:
newton_act_labels = [list(range(n_act_dim)) for _ in range(n_obs_dim)]
newton_act_labels = np.array(newton_act_labels).astype(int)

reverse_inner = lambda x: [x[0]] + x[1:-1][::-1] + [x[-1]]

aristotle_act_labels = [list(reversed(range(n_act_dim))) for _ in range(n_obs_dim)]
aristotle_act_labels = np.array(aristotle_act_labels).astype(int)

In [ ]:
all_goals = list(zip(*[x.ravel() for x in np.meshgrid(
    np.arange(0, gw_size-2, 1), np.arange(0, gw_size-2, 1))]))
train_goals = [all_goals[i] for i in np.random.choice(list(range(len(all_goals))), n_train_tasks, replace=False)]
train_goals = np.array(train_goals)
train_goals += 1

In [ ]:
plt.scatter(train_goals[:, 0], train_goals[:, 1], linewidth=0, color='gray', s=100, marker='*')
plt.xlim([-0.5, gw_size-0.5])
plt.ylim([-0.5, gw_size-0.5])
plt.show()

In [ ]:
with open(os.path.join(data_dir, 'train_goals.pkl'), 'wb') as f:
  pickle.dump(train_goals, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
with open(os.path.join(data_dir, 'train_goals.pkl'), 'rb') as f:
  train_goals = pickle.load(f)

In [ ]:
def make_reward_func(goal):
  def pos_from_obs(obs):
    x = obs // gw_size
    y = obs % gw_size
    return np.array([x, y])
  
  def reward_shaping(obs):
    return -np.linalg.norm((pos_from_obs(obs) - goal) / gw_size)

  def reward_func(prev_obs, action, obs):
    pos = pos_from_obs(obs)
    if (pos < 0).any() or (pos >= gw_size).any():
      r = crash_rew_penalty
    elif (pos == goal).all():
      r = succ_rew_bonus
    else:
      r = 0
    r += gamma * reward_shaping(obs) - reward_shaping(prev_obs)
    return r
  
  return reward_func

In [ ]:
class GridWorldNav(gym.Env):
  metadata = {
    'render.modes': ['human']
  }
  
  def __init__(
      self, 
      act_labels=None,
      max_ep_len=max_ep_len, 
      reward_func=None,
      goal=None
    ):
    self.observation_space = spaces.Discrete(n_obs_dim)
    self.action_space = spaces.Discrete(n_act_dim)
    
    self.pos = None
    self.curr_step = None
    self.viewer = None
    self.curr_obs = None
    self.next_obs = None
    
    self.succ_rew_bonus = succ_rew_bonus
    self.max_ep_len = max_ep_len
    self.reward_func = reward_func
    self.act_labels = act_labels
    self.goal = goal
    
    self.R = np.zeros((n_obs_dim, n_act_dim, n_obs_dim))
    for s in range(n_obs_dim):
      for sp in range(n_obs_dim):
        self.R[s, :, sp] = self.reward_func(s, None, sp)
          
    self.T = np.zeros((n_obs_dim, n_act_dim, n_obs_dim))
    for s in range(n_obs_dim-1):
      x = s // gw_size
      y = s % gw_size
      self.T[s, self.act_labels[s, 0], x*gw_size+(y-1) if y > 0 else -1] = 1
      self.T[s, self.act_labels[s, 1], x*gw_size+(y+1) if y < gw_size-1 else -1] = 1
      self.T[s, self.act_labels[s, 2], (x-1)*gw_size+y if x > 0 else -1] = 1
      self.T[s, self.act_labels[s, 3], (x+1)*gw_size+y if x < gw_size-1 else -1] = 1
    self.T[-1, :, -1] = 1
    
  def _obs(self):
    self.curr_obs = int(self.pos[0]*gw_size + self.pos[1])
    if self.curr_obs < 0 or self.curr_obs >= gw_size**2:
      self.curr_obs = gw_size**2
    return self.curr_obs

  def _step(self, action):
    if self.next_obs is None:
      if action == self.act_labels[self.curr_obs, 0]: # left
        self.pos[1] -= 1
      elif action == self.act_labels[self.curr_obs, 1]: # right
        self.pos[1] += 1
      elif action == self.act_labels[self.curr_obs, 2]: # up
        self.pos[0] -= 1
      elif action == self.act_labels[self.curr_obs, 3]: # down
        self.pos[0] += 1
      else:
        raise ValueError('invalid action')
    else:
      self.pos = np.array([self.next_obs // gw_size, self.next_obs % gw_size])
          
    self.curr_step += 1
    succ = (self.pos == self.goal).all()
    oob = (self.pos < 0).any() or (self.pos >= gw_size).any()
    oot = self.curr_step >= self.max_ep_len
    
    obs = self._obs()
    r = self.reward_func(self.prev_obs, action, obs)
    done = oot or succ or oob
    info = {}
    self.prev_obs = obs
    
    return obs, r, done, info
    
  def _reset(self):
    pos = (np.random.choice(gw_size**2-1) + self.goal[0]*gw_size + self.goal[1]) % (gw_size**2)
    self.pos = np.array([pos // gw_size, pos % gw_size])
    
    self.curr_step = 0
    self.prev_obs = self._obs()
    self.next_obs = None
    return self.prev_obs
  
  def _render(self, mode='human', close=False):
    if close:
      if self.viewer is not None:
        self.viewer.close()
        self.viewer = None
      return
    
    if self.viewer is None:
      self.viewer = rendering.SimpleImageViewer()
    
    fig = plt.figure()
    canvas = FigureCanvas(fig)
    
    plt.scatter([self.goal[0]], [self.goal[1]], color='gray', linewidth=0, alpha=0.75, marker='*')
    plt.scatter([self.pos[0]], [self.pos[1]], color='orange', linewidth=0, alpha=0.75)
    plt.xlim([-1, gw_size+1])
    plt.ylim([-1, gw_size+1])
    plt.axis('off')
    
    agg = canvas.switch_backends(FigureCanvas)
    agg.draw()
    width, height = fig.get_size_inches() * fig.get_dpi()
    self.viewer.imshow(np.fromstring(agg.tostring_rgb(), dtype='uint8').reshape(int(height), int(width), 3))
    plt.close()

In [ ]:
train_reward_funcs = [make_reward_func(goal) for goal in train_goals]
train_newton_envs = [GridWorldNav(reward_func=r, goal=train_goals[i], act_labels=newton_act_labels) for i, r in enumerate(train_reward_funcs)]
train_aristotle_envs = [GridWorldNav(reward_func=r, goal=train_goals[i], act_labels=aristotle_act_labels) for i, r in enumerate(train_reward_funcs)]

In [ ]:
def run_ep(policy, env, max_ep_len=max_ep_len, render=False, task_idx=None):
  obs = env.reset()
  done = False
  totalr = 0.
  prev_obs = obs
  rollout = []
  for step_idx in range(max_ep_len+1):
    if done:
      break
    action = policy(obs)
    obs, r, done, info = env.step(action)
    rollout.append((prev_obs, action, r, obs, float(done), task_idx))
    prev_obs = obs
    if render:
      env.render()
    totalr += r
  return rollout

In [ ]:
def make_aristotle_pilot_policy(goal):
  gx, gy = goal
  def aristotle_pilot_policy(obs):
    x = obs // gw_size
    y = obs % gw_size
    up = gx<x
    down = gx>x
    left = gy<y
    right = gy>y
    lr = left or right
    ud = up or down
    if lr and (not ud or np.random.random() < 0.5):
      if left:
        return aristotle_act_labels[obs, 0]
      elif right:
        return aristotle_act_labels[obs, 1]
    elif ud:
      if up:
        return aristotle_act_labels[obs, 2]
      elif down:
        return aristotle_act_labels[obs, 3]
    return aristotle_act_labels[obs, 0]
  return aristotle_pilot_policy

train agent with tabular Q-learning

In [ ]:
def tabsoftq_iter(R, T, maxiter=10000, verbose=True, Q_init=None, learning_rate=1, ftol=1e-32):
  n, m = R.shape[:2]
  Q = np.zeros((n, m)) if Q_init is None else copy(Q_init)
  prevQ = copy(Q)
  if verbose:
    diffs = []
  for iter_idx in range(maxiter):
    V = logsumexp(prevQ, axis=1)
    V_broad = V.reshape((1, 1, n))
    Q = np.sum(T * (R + gamma * V_broad), axis=2)
    Q = (1 - learning_rate) * prevQ + learning_rate * Q
    diff = np.mean((Q - prevQ)**2)/(np.std(Q)**2)
    if verbose:
      diffs.append(diff)
    if diff < ftol:
      break
    prevQ = copy(Q)
  if verbose:
    plt.xlabel('Number of Iterations')
    plt.ylabel('Avg. Squared Bellman Error')
    plt.title('Soft Q Iteration')
    plt.plot(diffs)
    plt.yscale('log')
    plt.show()
  return Q

In [ ]:
def tabsoftq_learn(env):
  R = env.unwrapped.R
  T = env.unwrapped.T
  return tabsoftq_iter(R, T)

In [ ]:
aristotle_softq_pilot_temp = 1
def make_tabsoftq_policy(Q):
  def tabsoftq_policy(obs):
    return np.argmax(aristotle_softq_pilot_temp*Q[obs, :] + np.random.gumbel(0, 1, n_act_dim))
  return tabsoftq_policy

In [ ]:
Q = np.stack([tabsoftq_learn(train_aristotle_envs[train_task_idx]) for train_task_idx in range(n_train_tasks)], axis=0)

In [ ]:
for i in range(n_train_tasks):
  plt.imshow(np.argmax(Q[i, :-1], axis=1).reshape((gw_size, gw_size)).T)
  plt.scatter(train_goals[i, 0], train_goals[i, 1], linewidth=0, color='gray', s=200, marker='*')
  plt.show()

In [ ]:
for i in range(n_train_tasks):
  plt.imshow(np.max(Q[i, :-1, :], axis=1).reshape((gw_size, gw_size)).T)
  plt.scatter(train_goals[i, 0], train_goals[i, 1], linewidth=0, color='gray', s=200, marker='*')
  plt.show()

In [ ]:
aristotle_tabsoftq_pilot_path = os.path.join(data_dir, 'train_aristotle_tabsoftq_pilots.pkl')

In [ ]:
with open(aristotle_tabsoftq_pilot_path, 'wb') as f:
  pickle.dump(Q, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
with open(aristotle_tabsoftq_pilot_path, 'rb') as f:
  Q = pickle.load(f)

In [ ]:
aristotle_pilot_policies = [make_tabsoftq_policy(Q[i]) for i in range(n_train_tasks)]

sanity-check envs, agents

In [ ]:
train_task_idx = 0

In [ ]:
run_ep(aristotle_pilot_policies[train_task_idx], train_aristotle_envs[train_task_idx], render=True)

In [ ]:
train_aristotle_envs[train_task_idx].close()

In [ ]:
run_ep(aristotle_pilot_policies[train_task_idx], train_newton_envs[train_task_idx], render=True)

In [ ]:
train_newton_envs[train_task_idx].close()

fit internal dynamics model

In [ ]:
n_train_rollouts_per_env = 100

In [ ]:
demo_rollouts = [[run_ep(aristotle_pilot_policies[train_task_idx], newton_env, render=False, task_idx=train_task_idx)
                  for _ in range(n_train_rollouts_per_env)] 
                 for train_task_idx, newton_env in enumerate(train_newton_envs)]

In [ ]:
with open(os.path.join(data_dir, 'aristotle_pilot_policy_demo_rollouts.pkl'), 'wb') as f:
  pickle.dump(demo_rollouts, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
with open(os.path.join(data_dir, 'aristotle_pilot_policy_demo_rollouts.pkl'), 'rb') as f:
  demo_rollouts = pickle.load(f)

In [ ]:
def build_mlp(
    input_placeholder,
    output_size,
    scope,
    n_layers=2,
    size=500,
    activation=tf.nn.relu,
    output_activation=None,
    reuse=False,
    kernel_initializer=tf.glorot_uniform_initializer()
  ):
  out = input_placeholder
  with tf.variable_scope(scope, reuse=reuse):
    for _ in range(n_layers):
      out = tf.layers.dense(out, size, activation=activation)
    out = tf.layers.dense(
      out, output_size, activation=output_activation,
      kernel_initializer=kernel_initializer)
  return out

In [ ]:
def vectorize_rollouts(rollouts):
  obs = []
  actions = []
  rewards = []
  next_obs = []
  dones = []
  task_idxes = []
  for rollout in rollouts:
    more_obs, more_actions, more_rewards, more_next_obs, more_dones, more_task_idxes = list(zip(*rollout))
    obs.extend(more_obs)
    actions.extend(more_actions)
    rewards.extend(more_rewards)
    next_obs.extend(more_next_obs)
    dones.extend(more_dones)
    task_idxes.extend(more_task_idxes)
  return np.array(obs), np.array(actions), np.array(rewards), np.array(next_obs), np.array(dones), np.array(task_idxes)

In [ ]:
vectorized_demo_rollouts = vectorize_rollouts(sum(demo_rollouts, []))

In [ ]:
demo_obs, demo_actions, demo_rewards, demo_next_obs, demo_done_masks, demo_task_idxes = vectorized_demo_rollouts
demo_example_idxes = list(range(len(demo_obs)))

In [ ]:
random.shuffle(demo_example_idxes)
n_train_demo_examples = int(0.9 * len(demo_example_idxes))
train_demo_example_idxes = demo_example_idxes[:n_train_demo_examples]
val_demo_example_idxes = demo_example_idxes[n_train_demo_examples:]
val_demo_batch = demo_obs[val_demo_example_idxes], demo_actions[val_demo_example_idxes], demo_next_obs[val_demo_example_idxes], demo_task_idxes[val_demo_example_idxes]

In [ ]:
def sample_batch(size):
  idxes = random.sample(train_demo_example_idxes, size)
  demo_batch = demo_obs[idxes], demo_actions[idxes], demo_next_obs[idxes], demo_task_idxes[idxes]
  return demo_batch

In [ ]:
gamma = 0.99
iterations = 1000
learning_rate = 1e-3
batch_size = 512
sq_td_err_penalty = 1e2

q_n_layers = 0
q_layer_size = None
q_activation = None
q_output_activation = None

n_layers = 0
layer_size = None
activation = None
output_activation = None

val_update_freq = 100
n_val_rollouts_per_env = 10

In [ ]:
r_scope = str(uuid.uuid4())
q_scope = str(uuid.uuid4())

In [ ]:
demo_obs_t_ph = tf.placeholder(tf.int32, [None])
demo_act_t_ph = tf.placeholder(tf.int32, [None])
demo_task_t_ph = tf.placeholder(tf.int32, [None])
demo_batch_size_ph = tf.placeholder(tf.int32)

In [ ]:
def featurize_obs(obs):
  return tf.one_hot(obs, n_obs_dim)

In [ ]:
demo_batch_idxes = tf.reshape(tf.range(0, demo_batch_size_ph, 1), [demo_batch_size_ph, 1])

demo_q_t = tf.stack([build_mlp(
  featurize_obs(demo_obs_t_ph), n_act_dim, q_scope+'-'+str(train_task_idx), 
  n_layers=q_n_layers, size=q_layer_size,
  activation=q_activation, output_activation=q_output_activation
) for train_task_idx in range(n_train_tasks)], axis=0)
demo_q_t = tf.gather_nd(
  demo_q_t, tf.concat([tf.expand_dims(demo_task_t_ph, 1), demo_batch_idxes], axis=1))

demo_act_idxes = tf.concat([demo_batch_idxes, tf.reshape(
  demo_act_t_ph, [demo_batch_size_ph, 1])], axis=1)
demo_act_val_t = tf.gather_nd(demo_q_t, demo_act_idxes)
state_val_t = tf.reduce_logsumexp(demo_q_t, axis=1)
act_log_likelihoods = demo_act_val_t - state_val_t

In [ ]:
neg_avg_log_likelihood = -tf.reduce_mean(act_log_likelihoods)

In [ ]:
obs_tp1_probs = tf.convert_to_tensor(train_aristotle_envs[0].unwrapped.T, dtype=tf.float32)

In [ ]:
q_tp1 = tf.stack([build_mlp(
  featurize_obs(tf.range(0, n_obs_dim, 1)), n_act_dim, q_scope+'-'+str(train_task_idx), n_layers=q_n_layers, size=q_layer_size,
  activation=q_activation, output_activation=q_output_activation, reuse=True
) for train_task_idx in range(n_train_tasks)], axis=0)

In [ ]:
v_tp1 = tf.reduce_logsumexp(q_tp1, axis=2)

all_rew = tf.stack([build_mlp(
  featurize_obs(tf.range(0, n_obs_dim, 1)), 1, r_scope+'-'+str(train_task_idx), 
  n_layers=n_layers, size=layer_size,
  activation=activation, output_activation=output_activation, 
  kernel_initializer=tf.zeros_initializer
) for train_task_idx in range(n_train_tasks)], axis=0)
all_rew = tf.stack([all_rew] * n_obs_dim, axis=1)
all_rew = tf.stack([all_rew] * n_act_dim, axis=2)
all_rew = tf.squeeze(all_rew, axis=[4])

v_tp1_broad = tf.reshape(v_tp1, [n_train_tasks, 1, 1, n_obs_dim])
obs_tp1_probs_broad = tf.expand_dims(obs_tp1_probs, 0)

exp_v_tp1 = tf.reduce_sum(obs_tp1_probs_broad * v_tp1_broad, axis=3)
exp_rew_t = tf.reduce_sum(obs_tp1_probs_broad * all_rew, axis=3)
target_t = exp_rew_t + gamma * exp_v_tp1

In [ ]:
q_t = tf.stack([build_mlp(
  featurize_obs(tf.range(0, n_obs_dim, 1)), n_act_dim, q_scope+'-'+str(train_task_idx), n_layers=q_n_layers, size=q_layer_size,
  activation=q_activation, output_activation=q_output_activation, reuse=True
) for train_task_idx in range(n_train_tasks)], axis=0)

In [ ]:
td_err = q_t - target_t

In [ ]:
sq_td_err = tf.reduce_mean(td_err**2)

In [ ]:
loss = neg_avg_log_likelihood + sq_td_err_penalty * sq_td_err

In [ ]:
update_op = tf.train.AdamOptimizer(learning_rate).minimize(loss)

In [ ]:
def compute_assisted_perf(R=None):
  if R is None:
    R = sess.run(all_rew)
  assisted_rollouts = [[] for _ in range(n_train_tasks)]
  for i in range(n_train_tasks):
    Q = tabsoftq_iter(R[i], train_newton_envs[i].unwrapped.T, maxiter=1000, verbose=False, Q_init=None, learning_rate=1, ftol=0)
    policy = make_tabsoftq_policy(Q)
    assisted_rollouts[i].extend([run_ep(policy, train_newton_envs[i], max_ep_len=max_ep_len, render=False, task_idx=i) for _ in range(n_val_rollouts_per_env)])
      
  assisted_rew = [np.mean([sum(x[2] for x in r) for r in rollouts]) for rollouts in assisted_rollouts]
  assisted_succ = [np.mean([1 if is_succ(r) else 0 for r in rollouts]) for rollouts in assisted_rollouts]
  assisted_crash = [np.mean([1 if is_crash(r) else 0 for r in rollouts]) for rollouts in assisted_rollouts]
  assisted_perf = {
    'assisted_rew': assisted_rew,
    'assisted_succ': assisted_succ,
    'assisted_crash': assisted_crash
  }
  return assisted_perf

In [ ]:
tf.global_variables_initializer().run(session=sess)

In [ ]:
n_iters = iterations * len(demo_obs) // batch_size
train_logs = {
  'loss_evals': [],
  'nll_evals': [],
  'ste_evals': [],
  'val_loss_evals': [],
  'val_nll_evals': [],
  'val_ste_evals': [],
  'assisted_rew_evals': [],
  'assisted_succ_evals': [],
  'assisted_crash_evals': []
}

In [ ]:
def compute_batch_loss(demo_batch, step=False, t=None):
  demo_batch_obs_t, demo_batch_act_t, demo_batch_obs_tp1, demo_batch_task_t = demo_batch
  
  feed_dict = {
    demo_obs_t_ph: demo_batch_obs_t,
    demo_act_t_ph: demo_batch_act_t,
    demo_task_t_ph: demo_batch_task_t,
    demo_batch_size_ph: demo_batch_obs_t.shape[0]
  }
  
  [loss_eval, neg_avg_log_likelihood_eval, sq_td_err_eval] = sess.run(
    [loss, neg_avg_log_likelihood, sq_td_err], feed_dict=feed_dict)
  
  if step:
    sess.run(update_op, feed_dict=feed_dict)
  
  d = {
    'loss': loss_eval,
    'nll': neg_avg_log_likelihood_eval,
    'ste': sq_td_err_eval
  }
  if not step:
    d.update(compute_assisted_perf())
  return d

In [ ]:
val_log = None
while len(train_logs['loss_evals']) < n_iters:
  demo_batch = sample_batch(batch_size)
  
  t = len(train_logs['loss_evals'])
  train_log = compute_batch_loss(demo_batch, step=True, t=t)
  if val_log is None or len(train_logs['loss_evals']) % val_update_freq == 0:
    val_log = compute_batch_loss(val_demo_batch, step=False, t=t)
  
  print('%d %d %f %f %f %f %f %f %f' % (
    t, n_iters, train_log['loss'],
    train_log['nll'], train_log['ste'], val_log['loss'],
    val_log['nll'], val_log['ste'], val_log['assisted_rew'])
  )
  
  for k, v in train_log.items():
    train_logs['%s_evals' % k].append(v)
  for k, v in val_log.items():
    train_logs['%s%s_evals' % ('val_' if k in ['loss', 'nll', 'ste'] else '', k)].append(v)

In [ ]:
for k in ['val_nll_evals', 'val_ste_evals']:
  plt.xlabel('Iterations')
  plt.ylabel(k.split('_')[1])
  plt.plot(train_logs[k])
  plt.show()

In [ ]:
plt.xlabel('Iterations')
plt.ylabel('Reward')
plt.axhline(y=np.mean(ideal_rew), linestyle='--', color='teal', label='Optimal')
plt.plot(train_logs['assisted_rew_evals'], color='orange', label='Ours')
plt.legend(loc='best')
plt.show()

In [ ]:
plt.xlabel('Iterations')
plt.ylabel('Success Rate')
plt.axhline(y=np.mean(ideal_succ), linestyle='--', color='teal', label='Optimal')
plt.plot(train_logs['assisted_succ_evals'], color='orange', label='Ours')
plt.ylim([-0.05, 1.05])
plt.legend(loc='best')
plt.show()

In [ ]:
plt.xlabel('Iterations')
plt.ylabel('Crash Rate')
plt.axhline(y=np.mean(ideal_crash), linestyle='--', color='teal', label='Optimal')
plt.plot(train_logs['assisted_crash_evals'], color='orange', label='Ours')
plt.ylim([-0.05, 1.05])
plt.legend(loc='best')
plt.show()

tabular MaxCausalEnt IRL (see https://arxiv.org/abs/1604.03912)

In [ ]:
def tabsoftq_grad_iter(R, Q, maxiter=1000, verbose=True, learning_rate=1, G_init=None, ftol=0):
  n = n_obs_dim
  m = n_act_dim
  
  P_broad = (np.exp(Q) / np.sum(np.exp(Q), axis=1)[:, np.newaxis]).reshape((n, m, 1))
    
  dR = np.zeros((n, m, n))
  states = np.arange(0, n_obs_dim, 1)
  dR[states, :, states] = 1
  
  T_broad = T.reshape((n, m, n, 1))
        
  G = np.zeros((n, m, n)) if G_init is None else G_init
  prevG = copy(G)
  if verbose:
    diffs = []
  for iter_idx in range(maxiter):
    expG = np.sum(P_broad * G, axis=1)
    expG_broad = expG.reshape((1, 1, n, n))
    G = dR + gamma * np.sum(T_broad * expG_broad, axis=2)
    G = (1 - learning_rate) * prevG + learning_rate * G
    
    diff = np.mean((G - prevG)**2)/(np.std(G)**2)
    if verbose:
      diffs.append(diff)
    if diff < ftol:
      break
    prevG = copy(G)
  
  if verbose:
    plt.xlabel('Number of Iterations')
    plt.ylabel('Avg. Squared Bellman Error')
    plt.title('Soft Q Gradient Iteration')
    plt.plot(diffs)
    plt.yscale('log')
    plt.show()
  
  expG = np.sum(P_broad * G, axis=1)
  expG_broad = expG.reshape((n, 1, n))
  return expG_broad - G

In [ ]:
def sample_batch(size):
  idxes = random.sample(train_demo_example_idxes, size)
  demo_batch = demo_obs[idxes], demo_actions[idxes], demo_task_idxes[idxes]
  return demo_batch

In [ ]:
def eval_loss(Q, task_idxes, obs, actions):
  loss = -np.mean(Q[task_idxes, obs, actions] - logsumexp(Q[task_idxes, obs, :], axis=1))
  return loss

In [ ]:
def eval_grad(R, Q, task_idxes, obs, actions):
  dR = np.stack([tabsoftq_grad_iter(
    R[i], Q[i], maxiter=tabsoftq_grad_iter_maxiter, verbose=tabsoftq_grad_iter_verbose, 
    ftol=tabsoftq_grad_iter_ftol) for i in range(n_train_tasks)], axis=0)
  dR = np.mean(dR[task_idxes, obs, actions], axis=0)
  return dR

In [ ]:
def R_s_to_sasp(R):
  return np.stack([np.stack([R] * n_act_dim, axis=0)] * n_obs_dim, axis=0)

In [ ]:
def Q_from_R(R, Q_inits=None):
  Q = np.stack([tabsoftq_iter(
    R_s_to_sasp(R[i]), T, Q_init=(Q_inits[i] if Q_inits is not None else None), 
    maxiter=tabsoftq_iter_maxiter, verbose=tabsoftq_iter_verbose, 
    ftol=tabsoftq_iter_ftol) for i in range(n_train_tasks)], axis=0)
  return Q

In [ ]:
def eval_loss_and_grad(R, task_idxes, obs, actions, Q_inits=None):
  Q = Q_from_R(R, Q_inits=Q_inits)
  dR = eval_grad(R, Q, task_idxes, obs, actions)
  loss = eval_loss(Q, task_idxes, obs, actions)
  return loss, dR, Q

In [ ]:
val_demo_obs, val_demo_actions, val_demo_next_obs, val_demo_task_idxes = val_demo_batch

In [ ]:
learning_rate = 1e-1
batch_size = len(train_demo_example_idxes)
maxiter = 100
tabsoftq_iter_ftol = 1e-32
tabsoftq_grad_iter_ftol = 1e-32
tabsoftq_iter_maxiter = 5000
tabsoftq_grad_iter_maxiter = 5000
tabsoftq_iter_verbose = False
tabsoftq_grad_iter_verbose = False

In [ ]:
train_logs = {
  'nll_evals': [],
  'val_nll_evals': [],
  'assisted_succ': [],
  'assisted_rew': [],
  'assisted_crash': []
}

In [ ]:
T = train_aristotle_envs[0].unwrapped.T

In [ ]:
R = np.random.random((n_train_tasks, n_obs_dim))
Q = None

In [ ]:
while len(train_logs['nll_evals']) < maxiter:
  batch_demo_obs, batch_demo_actions, batch_demo_task_idxes = sample_batch(batch_size)
  nll_eval, dR, Q = eval_loss_and_grad(
    R, batch_demo_task_idxes, batch_demo_obs, batch_demo_actions, Q_inits=Q)
  R -= learning_rate * dR
  
  val_nll_eval = eval_loss(Q, val_demo_task_idxes, val_demo_obs, val_demo_actions)
  assisted_perf = compute_assisted_perf(
    np.stack([R_s_to_sasp(R[i]) for i in range(n_train_tasks)], axis=0))
  
  print('%d %f %f %f' % (
    len(train_logs['nll_evals']), nll_eval, val_nll_eval, assisted_perf['assisted_rew']))
  train_logs['nll_evals'].append(nll_eval)
  train_logs['val_nll_evals'].append(val_nll_eval)
  for k, v in assisted_perf.items():
    train_logs[k].append(v)

In [ ]:
for k in ['val_nll_evals', 'nll_evals']:
  plt.xlabel('Iterations')
  plt.ylabel(k.replace('_', ' '))
  plt.plot(train_logs[k])
  plt.show()

In [ ]:
plt.xlabel('Iterations')
plt.ylabel('Reward')
plt.axhline(y=np.mean(ideal_rew), linestyle='--', color='teal', label='Optimal')
plt.plot(train_logs['assisted_rew'], color='orange', label='Ours')
plt.legend(loc='best')
plt.show()

In [ ]:
plt.xlabel('Iterations')
plt.ylabel('Success Rate')
plt.axhline(y=np.mean(ideal_succ), linestyle='--', color='teal', label='Optimal')
plt.plot(train_logs['assisted_succ'], color='orange', label='Ours')
plt.ylim([-0.05, 1.05])
plt.legend(loc='best')
plt.show()

In [ ]:
plt.xlabel('Iterations')
plt.ylabel('Crash Rate')
plt.axhline(y=np.mean(ideal_crash), linestyle='--', color='teal', label='Optimal')
plt.plot(train_logs['assisted_crash'], color='orange', label='Ours')
plt.ylim([-0.05, 1.05])
plt.legend(loc='best')
plt.show()

In [ ]:
plt.imshow(R[0, :-1].reshape((gw_size, gw_size)))
plt.show()

In [ ]:
plt.imshow(np.max(Q[0, :-1, :], axis=1).reshape((gw_size, gw_size)))
plt.show()

use learned internal dynamics model, repeat with ten different random seeds

In [ ]:
master_train_logs = []

In [ ]:
for _ in range(10):
  train_logs = {
    'nll_evals': [],
    'val_nll_evals': [],
    'assisted_succ': [],
    'assisted_rew': [],
    'assisted_crash': []
  }
  
  R = np.random.random((n_train_tasks, n_obs_dim))
  Q = None
  
  while len(train_logs['nll_evals']) < maxiter:
    batch_demo_obs, batch_demo_actions, batch_demo_task_idxes = sample_batch(batch_size)
    nll_eval, dR, Q = eval_loss_and_grad(
      R, batch_demo_task_idxes, batch_demo_obs, batch_demo_actions, Q_inits=Q)
    R -= learning_rate * dR

    val_nll_eval = eval_loss(Q, val_demo_task_idxes, val_demo_obs, val_demo_actions)
    assisted_perf = compute_assisted_perf(
      np.stack([R_s_to_sasp(R[i]) for i in range(n_train_tasks)], axis=0))

    print('%d %f %f %f %f' % (
      len(train_logs['nll_evals']), nll_eval, val_nll_eval, np.max(
        assisted_perf['assisted_rew']), np.min(assisted_perf['assisted_rew'])))
    train_logs['nll_evals'].append(nll_eval)
    train_logs['val_nll_evals'].append(val_nll_eval)
    for k, v in assisted_perf.items():
      train_logs[k].append(v)

  master_train_logs.append(train_logs)

In [ ]:
with open(os.path.join(data_dir, 'serd_aristotle_master_train_logs.pkl'), 'wb') as f:
  pickle.dump(master_train_logs, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
def make_ideal_env():
  test_goal = np.random.choice(gw_size, 2)
  test_reward_func = make_reward_func(test_goal)
  test_aristotle_pilot_policy = make_aristotle_pilot_policy(test_goal)
  unassisted_env = GridWorldNav(
    act_labels=aristotle_act_labels, reward_func=test_reward_func, goal=test_goal)
  return test_aristotle_pilot_policy, unassisted_env

In [ ]:
def make_rand_env():
  test_goal = np.random.choice(gw_size, 2)
  test_reward_func = make_reward_func(test_goal)
  test_aristotle_pilot_policy = (lambda _: np.random.choice(list(range(n_act_dim))))
  unassisted_env = GridWorldNav(
    act_labels=aristotle_act_labels, reward_func=test_reward_func, goal=test_goal)
  return test_aristotle_pilot_policy, unassisted_env

In [ ]:
n_eval_rollouts = 100

In [ ]:
ideal_rollouts = [run_ep(*make_ideal_env(), render=False) for _ in range(n_eval_rollouts)]

In [ ]:
with open(os.path.join(data_dir, 'aristotle_pilot_policy_ideal_rollouts.pkl'), 'wb') as f:
  pickle.dump(ideal_rollouts, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
with open(os.path.join(data_dir, 'aristotle_pilot_policy_ideal_rollouts.pkl'), 'rb') as f:
  ideal_rollouts = pickle.load(f)

In [ ]:
rand_rollouts = [run_ep(*make_rand_env(), render=False) for _ in range(n_eval_rollouts)]

In [ ]:
with open(os.path.join(data_dir, 'rand_pilot_policy_unassisted_rollouts.pkl'), 'wb') as f:
  pickle.dump(rand_rollouts, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
with open(os.path.join(data_dir, 'rand_pilot_policy_unassisted_rollouts.pkl'), 'rb') as f:
  rand_rollouts = pickle.load(f)

In [ ]:
ideal_rew = [sum(x[2] for x in r) for r in ideal_rollouts]

In [ ]:
rand_rew = [sum(x[2] for x in r) for r in rand_rollouts]

In [ ]:
np.mean(ideal_rew)

In [ ]:
np.mean(rand_rew)

In [ ]:
ideal_succ = [1 if is_succ(r) else 0 for r in ideal_rollouts]

In [ ]:
rand_succ = [1 if is_succ(r) else 0 for r in rand_rollouts]

In [ ]:
np.mean(ideal_succ)

In [ ]:
np.mean(rand_succ)

In [ ]:
ideal_crash = [1 if is_crash(r) else 0 for r in ideal_rollouts]

In [ ]:
rand_crash = [1 if is_crash(r) else 0 for r in rand_rollouts]

In [ ]:
np.mean(ideal_crash)

In [ ]:
np.mean(rand_crash)

viz master logs

In [ ]:
smooth_win = 100
def moving_avg(d, n=smooth_win):
  s = np.concatenate((np.zeros(1), np.cumsum(d).astype(float)))
  return (s[n:] - s[:-n]) / n

In [ ]:
traj_col_means = lambda x: np.nanmean(x, axis=0)
traj_col_stderrs = lambda x: np.nanstd(x, axis=0) / np.sqrt(
  np.count_nonzero(~np.isnan(x), axis=0))
r_mins = lambda x: traj_col_means(x) - traj_col_stderrs(x)
r_maxs = lambda x: traj_col_means(x) + traj_col_stderrs(x)

In [ ]:
def plot_fill(R, color, label, linestyle):
  x = range(R.shape[1] - (smooth_win - 1))
  y1 = moving_avg(r_mins(R), n=smooth_win)
  y2 = moving_avg(r_maxs(R), n=smooth_win)
  plt.fill_between(
    x, y1, y2, where=y2 >= y1, interpolate=True, facecolor=color, alpha=0.5)
  plt.plot(moving_avg(traj_col_means(R), n=smooth_win), color=color, label=label, linestyle=linestyle)

In [ ]:
with open(os.path.join(data_dir, 'serd_aristotle_master_train_logs.pkl'), 'rb') as f:
  serd_aristotle_master_train_logs = pickle.load(f)
  
with open(os.path.join(data_dir, 'serd_newton_master_train_logs.pkl'), 'rb') as f:
  serd_newton_master_train_logs = pickle.load(f)

In [ ]:
def rew_vs_iter_of_logs(master_train_logs):
  n_reps = len(master_train_logs)
  max_iter = max(len(
    train_logs['assisted_rew']) for train_logs in master_train_logs)
  R = np.zeros((n_reps, max_iter, n_train_tasks))
  R[:, :, :] = np.nan
  for i, train_logs in enumerate(master_train_logs):
    rews = train_logs['assisted_rew']
    R[i, :len(rews), :] = rews
  return R.swapaxes(1, 2).reshape((n_reps * n_train_tasks, max_iter))

In [ ]:
R_aristotle = rew_vs_iter_of_logs(serd_aristotle_master_train_logs)
R_newton = rew_vs_iter_of_logs(serd_newton_master_train_logs)

In [ ]:
mpl.rcParams.update({'font.size': 20})
smooth_win = 1

In [ ]:
plt.xlabel('Number of Gradient Steps')
plt.ylabel('True Reward')
plt.title('Learning Rewards from Misguided Demos')

plot_fill(R_aristotle, 'orange', 'IRL + Our Method', '-')
plot_fill(R_newton, 'teal', 'Tabular MaxCausalEnt IRL', ':')

plt.axhline(y=np.mean(rand_rew), linestyle='--', color='gray', label='Random Policy')

plt.xlim([0, 50])
plt.legend(loc='best', framealpha=0.5)
plt.savefig(os.path.join(data_dir, 'succ-vs-iter-serd.pdf'), bbox_inches='tight')
plt.show()